In [385]:
import sqlite3
import pandas as pd

In [386]:
# Created connection.
database_path = r"database.sqlite"

conn = sqlite3.connect(database_path)

In [387]:
# Creating dataframes from the tbales

df_country = pd.read_sql_query("SELECT * FROM country", conn)
df_league = pd.read_sql_query("SELECT * FROM league", conn)
df_match = pd.read_sql_query("SELECT * FROM match", conn)
df_player = pd.read_sql_query("SELECT * FROM player", conn)
df_player_attributes = pd.read_sql_query("SELECT * FROM player_attributes", conn)
df_team = pd.read_sql_query("SELECT * FROM team", conn)
df_team_attributes = pd.read_sql_query("SELECT * FROM team_attributes", conn)
df_sqlite_sequence = pd.read_sql_query("SELECT * FROM sqlite_sequence", conn)


In [388]:
# Deleting Duplicate values from the table

df_country = df_country.drop_duplicates()
df_league = df_league.drop_duplicates()
df_match = df_match.drop_duplicates()
df_player = df_player.drop_duplicates()
df_player_attributes = df_player_attributes.drop_duplicates()
df_team = df_team.drop_duplicates()
df_team_attributes = df_team_attributes.drop_duplicates()
df_sqlite_sequence = df_sqlite_sequence.drop_duplicates()

In [389]:
# Concate all the tables in one
# Note: removed and merged by analyzed the all table relationship by zeel. so, before making change make sure to communicate eith zeel

# merging country table
df_merged_all_table = pd.merge(left=df_league, right=df_country, how='outer', left_on='country_id', right_on='id')
df_merged_all_table = df_merged_all_table.drop(["id_x", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "league_name"
df_merged_all_table.columns.values[1] = "league_id"
df_merged_all_table.columns.values[2] = "country_name"

# -----------------------------------------------------------------

# merging league table
df_merged_all_table = df_match.merge(df_merged_all_table, how='outer', left_on='league_id', right_on='league_id')
df_merged_all_table = df_merged_all_table.drop(["league_id", "country_id"], axis=1)
df_merged_all_table.columns.values[0] = "match_id" 
df_merged_all_table = df_merged_all_table.drop(["match_id"], axis=1)

# -----------------------------------------------------------------

# merging team_attribute table with team table
df_merged_team_table = pd.merge(left=df_team, right=df_team_attributes, how='outer', left_on=['team_api_id'], right_on=['team_api_id'])
df_merged_team_table = df_merged_team_table.drop(["id_x", "id_y", "team_fifa_api_id_y"], axis=1)
df_merged_team_table.columns.values[1] = "team_fifa_api_id" 

df_merged_team_table.drop_duplicates()

# -----------------------------------------------------------------

# merging merged team table
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['home_team_api_id'], right_on=['team_api_id'])
df_merged_all_table = df_merged_all_table.drop(["date_y"], axis=1)
df_merged_all_table.columns.values[2] = "date" 
df_merged_all_table = df_merged_all_table.merge(df_merged_team_table, how='outer', left_on=['away_team_api_id'], right_on=['team_api_id'])
df_merged_all_table.columns.values[2] = "date" 

for i in range(26):
    df_merged_all_table = df_merged_all_table.drop([df_merged_all_table.columns[-1]], axis=1)

for i in range(26):
    df_merged_all_table.columns.values[-1 - i] = df_merged_all_table.columns[-1 - i].split("_x")[0]

# -----------------------------------------------------------------

# merging player_attribute table with player table
df_merged_player_table = pd.merge(left=df_player, right=df_player_attributes, how='outer', left_on=['player_api_id'], right_on=['player_api_id'])
df_merged_player_table = df_merged_player_table.drop(["id_x", "id_y", "player_fifa_api_id_y"], axis=1)
df_merged_player_table.columns.values[2] = "player_fifa_api_id" 

df_merged_player_table.drop_duplicates() 

# -----------------------------------------------------------------

# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(["team_fifa_api_id", "team_api_id", "match_api_id"], axis=1)

# -----------------------------------------------------------------

df_merged_all_table.drop_duplicates(inplace=True)

In [390]:
print(*df_merged_all_table.columns)

season stage date home_team_api_id away_team_api_id home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 home_player_1 home_player_2 home_player_3 home_player_4 home_player_5 home_player_6 home_player_7 home_player_8 home_player_9 home_player_10 home_player_11 away_player_1 away_player_2 away_player_3 away_player_4 away_player_5 away_player_6 away_playe

In [391]:
# combining total goal of a respected team

home_total_goal = pd.read_sql_query("select home_team_api_id, sum(home_team_goal) as home_total_goal from match group by home_team_api_id", conn)
away_total_goal = pd.read_sql_query("select away_team_api_id, sum(away_team_goal) as away_total_goal from match group by away_team_api_id", conn)

In [392]:
home_total_goal.shape

(299, 2)

In [393]:
# merging both the home and away total goal to the match table

df_merged_all_table = pd.merge(df_merged_all_table, home_total_goal, on='home_team_api_id')
df_merged_all_table = pd.merge(df_merged_all_table, away_total_goal, on='away_team_api_id')

In [394]:
df_match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [395]:
df_merged_player_table.head()

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [396]:
# getting overall rating of player

player_overall_rating = pd.read_sql_query("select * from (select player_api_id , max(overall_rating) as overall_rating from Player_Attributes group by player_api_id) as p where p.overall_rating is not null", conn)


In [397]:
player_overall_rating

,player_api_id,overall_rating
0,2625,63
1,2752,72
2,2768,75
3,2770,74
4,2790,73
...,...,...
11055,744907,53
11056,746419,59
11057,748432,58
11058,750435,60


In [398]:
print(*df_merged_all_table.columns)

season stage date home_team_api_id away_team_api_id home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 home_player_1 home_player_2 home_player_3 home_player_4 home_player_5 home_player_6 home_player_7 home_player_8 home_player_9 home_player_10 home_player_11 away_player_1 away_player_2 away_player_3 away_player_4 away_player_5 away_player_6 away_playe

In [399]:
for i in range(1,12):
    df_merged_all_table = df_merged_all_table.merge(player_overall_rating, how='left', left_on='home_player_' + str(i), right_on='player_api_id')
    df_merged_all_table = df_merged_all_table.drop(['home_player_' + str(i), "player_api_id"], axis=1)
    df_merged_all_table.columns.values[-1] = 'home_player_' + str(i) 

for i in range(1,12):
    df_merged_all_table = df_merged_all_table.merge(player_overall_rating, how='left', left_on='away_player_' + str(i), right_on='player_api_id')
    df_merged_all_table = df_merged_all_table.drop(['away_player_' + str(i), "player_api_id"], axis=1)
    df_merged_all_table.columns.values[-1] = 'away_player_' + str(i) 

df_merged_all_table = df_merged_all_table.drop([], axis=1)

In [400]:
print(*df_merged_all_table.columns)

season stage date home_team_api_id away_team_api_id home_team_goal away_team_goal home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11 goal shoton shotoff foulcommit card cross corner possession B365H B365D B365A BWH BWD BWA IWH IWD IWA LBH LBD LBA PSH PSD PSA WHH WHD WHA SJH SJD SJA VCH VCD VCA GBH GBD GBA BSH BSD BSA league_name country_name team_long_name team_short_name buildUpP

In [401]:
temp_list = "season stage date home_team_api_id away_team_api_id home_player_X1 home_player_X2 home_player_X3 home_player_X4 home_player_X5 home_player_X6 home_player_X7 home_player_X8 home_player_X9 home_player_X10 home_player_X11 away_player_X1 away_player_X2 away_player_X3 away_player_X4 away_player_X5 away_player_X6 away_player_X7 away_player_X8 away_player_X9 away_player_X10 away_player_X11 home_player_Y1 home_player_Y2 home_player_Y3 home_player_Y4 home_player_Y5 home_player_Y6 home_player_Y7 home_player_Y8 home_player_Y9 home_player_Y10 home_player_Y11 away_player_Y1 away_player_Y2 away_player_Y3 away_player_Y4 away_player_Y5 away_player_Y6 away_player_Y7 away_player_Y8 away_player_Y9 away_player_Y10 away_player_Y11".split(" ")

In [402]:
temp_list

['season',
 'stage',
 'date',
 'home_team_api_id',
 'away_team_api_id',
 'home_player_X1',
 'home_player_X2',
 'home_player_X3',
 'home_player_X4',
 'home_player_X5',
 'home_player_X6',
 'home_player_X7',
 'home_player_X8',
 'home_player_X9',
 'home_player_X10',
 'home_player_X11',
 'away_player_X1',
 'away_player_X2',
 'away_player_X3',
 'away_player_X4',
 'away_player_X5',
 'away_player_X6',
 'away_player_X7',
 'away_player_X8',
 'away_player_X9',
 'away_player_X10',
 'away_player_X11',
 'home_player_Y1',
 'home_player_Y2',
 'home_player_Y3',
 'home_player_Y4',
 'home_player_Y5',
 'home_player_Y6',
 'home_player_Y7',
 'home_player_Y8',
 'home_player_Y9',
 'home_player_Y10',
 'home_player_Y11',
 'away_player_Y1',
 'away_player_Y2',
 'away_player_Y3',
 'away_player_Y4',
 'away_player_Y5',
 'away_player_Y6',
 'away_player_Y7',
 'away_player_Y8',
 'away_player_Y9',
 'away_player_Y10',
 'away_player_Y11']

In [403]:
# droping columns with contains id (Converting n3 form to n1 form)
# df_merged_all_table = df_merged_all_table.drop(columns=["team_fifa_api_id", "team_api_id", "match_api_id"])

In [404]:
df_merged_all_table.columns

Index(['season', 'stage', 'date', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2',
       'home_player_X3',
       ...
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11'],
      dtype='object', length=138)

In [405]:
df_merged_all_table = df_merged_all_table.drop(columns= temp_list)

In [406]:
df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H', 'B365D',
       'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH',
       'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'league_name',
       'country_name', 'team_long_name', 'team_short_name', 'buildUpPlaySpeed',
       'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 

In [407]:
# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(['league_name',
       'country_name', 'team_long_name', 'team_short_name', 'buildUpPlaySpeed',
       'buildUpPlaySpeedClass', 'buildUpPlayDribbling',
       'buildUpPlayDribblingClass', 'buildUpPlayPassing',
       'buildUpPlayPassingClass', 'buildUpPlayPositioningClass',
       'chanceCreationPassing', 'chanceCreationPassingClass',
       'chanceCreationCrossing', 'chanceCreationCrossingClass',
       'chanceCreationShooting', 'chanceCreationShootingClass',
       'chanceCreationPositioningClass', 'defencePressure',
       'defencePressureClass', 'defenceAggression', 'defenceAggressionClass',
       'defenceTeamWidth', 'defenceTeamWidthClass', 'defenceDefenderLineClass', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession'], axis=1)

df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_total_goal',
       'away_total_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [408]:
df_merged_all_table.columns

Index(['home_team_goal', 'away_team_goal', 'B365H', 'B365D', 'B365A', 'BWH',
       'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',
       'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA',
       'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'home_total_goal',
       'away_total_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [409]:
df_merged_all_table.sample()

,home_team_goal,away_team_goal,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
18918,3,2,8.5,4.75,1.44,8.25,4.2,1.42,7.0,4.0,...,83.0,80.0,82.0,83.0,86.0,82.0,86.0,88.0,88.0,85.0


In [413]:
# generating target variable
def get_match_label(match):
    home_goals = match['home_team_goal']
    away_goals = match['away_team_goal']
    #Identify match label
    if home_goals > away_goals:
        return "HomeWin"
    elif home_goals == away_goals:
        return "Draw"
    elif home_goals < away_goals:
        return "AwayWin"

df_merged_all_table['target'] = df_merged_all_table.apply(get_match_label, axis=1)

In [414]:
# droping columns with contains id (Converting n3 form to n1 form)
df_merged_all_table = df_merged_all_table.drop(['home_team_goal',
       'away_team_goal'], axis=1)

df_merged_all_table.columns

Index(['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
       'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
       'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD',
       'BSA', 'home_total_goal', 'away_total_goal', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11', 'target'],
      dtype='object')

In [411]:
# Converting dataframe into csv for modeling

df_merged_all_table.to_pickle('merged_all_table.csv' + ".pkl", compression="bz2")